# Tutorial 5: Joining the Ledgers
## Merging DataFrames in Pandas

---

### 📺 Video Walkthrough

Watch the video tutorial for this notebook: [YouTube Link](#)

---

### From the Archives: A Problem of Cross-Reference

*The apprentice archivist approached my desk with a question that revealed how little she understood about our work.*

*"I have the catch records," she said, "and I can see that crew CRW006 caught creature CR005. But which crew is CRW006? And what creature is CR005?"*

*"That," I told her, "is why we have multiple ledgers. The catch record tells you what happened. The crew registry tells you who did it. The creature catalog tells you what was caught. Your job is to connect them."*

*She stared at the three separate books on my desk.*

*"The Boss knew how to bring separate things together," I said. "Learn to do the same with data, and you might survive this desk."*

*—From the training notes of Chief Archivist Mink*

---

## What You Will Learn

In this tutorial, you will learn to:

1. Understand why data is split across multiple tables
2. Use `pd.merge()` to join DataFrames
3. Understand different join types (inner, left, right, outer)
4. Handle mismatched keys
5. Join on multiple columns

By the end, you will be able to answer questions like:
- Which crew leader has caught the most valuable specimens?
- What's the average danger rating of creatures caught by each crew?
- Which sectors have the highest-value catches, and what creatures live there?

---

In [ ]:
import pandas as pd

# Load data from GitHub
BASE_URL = "https://raw.githubusercontent.com/YOUR_USERNAME/yeller-quarry-data-science/main/data/"

creatures = pd.read_csv(BASE_URL + "creatures.csv")
crews = pd.read_csv(BASE_URL + "crews.csv")
catches = pd.read_csv(BASE_URL + "catches.csv")
traps = pd.read_csv(BASE_URL + "traps.csv")

# Add total_value to catches
catches['total_value'] = catches['quantity'] * catches['price_per_unit']

print("Data loaded from the Capital Archives.")
print(f"  Creatures: {len(creatures)} species")
print(f"  Crews: {len(crews)} crews")
print(f"  Catches: {len(catches)} records")
print(f"  Traps: {len(traps)} deployments")

---

## Part 1: Why Multiple Tables?

Look at a catch record:

In [ ]:
catches.iloc[25]

This record tells us *something* was caught, but uses codes:
- `crew_id`: CRW006 — but who is that?
- `creature_id`: CR005 — but what creature?

The answers are in other tables. This is called **normalization**—storing each type of information once, then linking with IDs.

### Why normalize?

1. **No redundancy**: The Boss's name doesn't appear 50 times in 50 catch records
2. **Easy updates**: If we learn a crew leader changed, we update one row
3. **Consistency**: No typos like "Maw Beast" vs "maw beast" vs "MawBeast"

But to *analyze* the data, we need to bring the tables back together.

---

## Part 2: Basic Merge

The `pd.merge()` function joins two DataFrames based on matching values in a column.

### Joining catches with creature names

In [ ]:
# Merge catches with creatures on creature_id
catches_with_creatures = pd.merge(
    catches,
    creatures[['creature_id', 'common_name', 'category', 'danger_rating']],
    on='creature_id',
    how='left'
)

catches_with_creatures.head(10)

Now we can see the actual creature names! The merge brought in `common_name`, `category`, and `danger_rating` from the creatures table.

### Understanding the merge parameters

```python
pd.merge(
    left,      # First DataFrame
    right,     # Second DataFrame  
    on='col',  # Column to match on (must exist in both)
    how='...'  # Type of join
)
```

---

## Part 3: Join Types

The `how` parameter controls what happens when values don't match.

| Join Type | Keeps |
|-----------|-------|
| `inner` | Only rows where key exists in BOTH tables |
| `left` | All rows from left table, matching rows from right |
| `right` | All rows from right table, matching rows from left |
| `outer` | All rows from both tables |

### Visual example

Imagine:
- Left table has creatures: A, B, C
- Right table has creatures: B, C, D

| Join | Result |
|------|--------|
| inner | B, C (only matches) |
| left | A, B, C (all from left) |
| right | B, C, D (all from right) |
| outer | A, B, C, D (everything) |

In [ ]:
# Let's see the difference
# Inner join: only creatures that appear in catches
inner_result = pd.merge(catches, creatures, on='creature_id', how='inner')
print(f"Inner join: {len(inner_result)} rows")

# Left join: all catches, with creature info where available
left_result = pd.merge(catches, creatures, on='creature_id', how='left')
print(f"Left join: {len(left_result)} rows")

# Right join: all creatures, with catch info where they were caught
right_result = pd.merge(catches, creatures, on='creature_id', how='right')
print(f"Right join: {len(right_result)} rows")

In [ ]:
# Right join shows creatures that were NEVER caught
never_caught = right_result[right_result['catch_id'].isna()]
print("Creatures never caught this season:")
never_caught[['creature_id', 'common_name', 'danger_rating']]

Several creatures have never been caught—including the Grimslew Fish and the Witch Creature. The archivists list them as "mythical." But the trappers who work the Grimslew Shore say the fish are real. They just don't survive the trip to the surface.

---

## Part 4: Adding Crew Information

Let's add crew names and leaders to our catch data.

In [ ]:
# First, look at the crews table
crews.head()

In [ ]:
# Merge catches with crews
catches_with_crews = pd.merge(
    catches,
    crews[['crew_id', 'crew_name', 'leader', 'specialty']],
    on='crew_id',
    how='left'
)

catches_with_crews[['catch_id', 'creature_id', 'crew_name', 'leader', 'total_value']].head(10)

Now we can see which leader was responsible for each catch.

---

## Part 5: Chaining Merges

We can merge multiple tables by chaining operations.

In [ ]:
# Create a comprehensive catch report
full_catch_report = (
    catches
    .merge(
        creatures[['creature_id', 'common_name', 'category', 'danger_rating', 'metal_content_pct']],
        on='creature_id',
        how='left'
    )
    .merge(
        crews[['crew_id', 'crew_name', 'leader']],
        on='crew_id',
        how='left'
    )
)

# Select the most useful columns
full_catch_report[['date', 'crew_name', 'leader', 'common_name', 'quantity', 'condition', 'total_value']].head(15)

Now we have a human-readable catch report. No more cryptic IDs!

---

## Part 6: Analysis with Merged Data

With the merged data, we can answer complex questions.

### Which leader has caught the most dangerous creatures (on average)?

In [ ]:
# Average danger rating by leader
full_catch_report.groupby('leader')['danger_rating'].mean().sort_values(ascending=False)

Captain Yuro of the Grimslew Salvagers leads in average danger rating. The Deep Tunnel Syndicate's Master Varek is close behind. These are the crews that work the most dangerous areas.

### Total revenue by leader

In [ ]:
# Total revenue by leader
leader_revenue = full_catch_report.groupby('leader').agg({
    'total_value': 'sum',
    'catch_id': 'count',
    'danger_rating': 'mean'
}).round(2)

leader_revenue.columns = ['total_revenue', 'num_catches', 'avg_danger']
leader_revenue['revenue_per_catch'] = (leader_revenue['total_revenue'] / leader_revenue['num_catches']).round(2)
leader_revenue.sort_values('total_revenue', ascending=False)

Guildmaster Thresh of the Miasto Trappers Guild has the highest total revenue—they're the oldest and most established crew. But Captain Yuro has the highest revenue *per catch* because wharvers are so valuable.

Note The Boss at the bottom with only 546 total revenue. The Redmane Expedition was cut short.

---

## Part 7: Merging with Different Column Names

Sometimes the join columns have different names in each table. Use `left_on` and `right_on`.

In [ ]:
# Example: if catches had 'cid' instead of 'creature_id'
# We would do:
# pd.merge(catches, creatures, left_on='cid', right_on='creature_id')

# Let's demonstrate by renaming temporarily
catches_renamed = catches.copy()
catches_renamed = catches_renamed.rename(columns={'creature_id': 'species_code'})

# Now merge with different column names
result = pd.merge(
    catches_renamed,
    creatures[['creature_id', 'common_name']],
    left_on='species_code',
    right_on='creature_id',
    how='left'
)

result[['species_code', 'creature_id', 'common_name']].head()

Notice both columns are kept. You can drop the duplicate if needed.

---

## Part 8: Merging on Multiple Columns

Sometimes you need to match on more than one column.

In [ ]:
# Create a hypothetical example: sector-specific crew assignments
# In reality, we might have a table showing which crew is assigned to which sector on which date

# For now, let's merge catches with traps to see which trap caught what
catches_with_traps = pd.merge(
    catches,
    traps[['trap_id', 'trap_type', 'maker', 'depth_placed_m']],
    on='trap_id',
    how='left'
)

catches_with_traps[['catch_id', 'trap_id', 'trap_type', 'maker', 'creature_id', 'quantity']].head(10)

Now we can see what type of trap caught each creature, and who made it.

### Which trap type catches the most valuable specimens?

In [ ]:
catches_with_traps.groupby('trap_type')['total_value'].agg(['sum', 'mean', 'count']).sort_values('mean', ascending=False)

Hooks have the highest average value per catch—they're used for deep-water extraction on Grimslew Shore where wharvers live. Cages also score high because they're used for live capture of premium specimens.

---

## Part 9: The Complete Picture

Let's build the ultimate merged dataset.

In [ ]:
# The master catch analysis table
master_catches = (
    catches
    .merge(creatures[['creature_id', 'common_name', 'category', 'danger_rating', 'metal_content_pct']], 
           on='creature_id', how='left')
    .merge(crews[['crew_id', 'crew_name', 'leader', 'specialty', 'base_pay_per_expedition']], 
           on='crew_id', how='left')
    .merge(traps[['trap_id', 'trap_type', 'maker', 'depth_placed_m']], 
           on='trap_id', how='left')
)

print(f"Master table: {len(master_catches)} catches with {len(master_catches.columns)} attributes")
master_catches.columns.tolist()

In [ ]:
# A rich view of catch CAT0027 - the Maw Beast
maw_beast_catch = master_catches[master_catches['creature_id'] == 'CR005']
maw_beast_catch[['date', 'common_name', 'danger_rating', 'crew_name', 'leader', 
                 'trap_type', 'depth_placed_m', 'condition', 'total_value']]

The Maw Beast. Caught dead by the Deep Tunnel Syndicate under Master Varek. Hook trap at 55 meters. 950 units—a record price.

This is what killed Truck. Or something like it.

---

## ✏️ Exercises

### Exercise 1: Sector Analysis

Which sector has the highest total revenue? Use the merged data to find out, and include the average danger rating of creatures caught in each sector.

In [ ]:
# Your code here
# Hint: groupby 'sector' on the master_catches table



### Exercise 2: The Boss's Traps

Filter the master_catches table to show only catches made using traps built by "The Boss". What creatures did her traps catch?

In [ ]:
# Your code here



### Exercise 3: Metal Content and Value

Is there a relationship between a creature's metal content and its catch value? Group by creature and calculate the average metal_content_pct and average price_per_unit.

In [ ]:
# Your code here



### Exercise 4: Crew Specialty Match

Each crew has a specialty (e.g., "deep tunnel extraction", "live capture"). Do crews catch more creatures that match their specialty? 

For example, does the live capture specialist crew catch more live specimens?

In [ ]:
# Your code here
# This is a challenging one! Try grouping by crew_name and condition



### Exercise 5: Uncaught Creatures Report

Create a report of all creatures that have never been caught this season. Include their danger rating, typical depth, and capital demand. Why might these creatures remain uncaught?

In [ ]:
# Your code here
# Hint: Use a right join and filter for null catch_ids



---

## 📋 Summary

In this tutorial, you learned:

| Concept | Code |
|---------|------|
| Basic merge | `pd.merge(left, right, on='column')` |
| Left join | `pd.merge(left, right, on='col', how='left')` |
| Right join | `pd.merge(left, right, on='col', how='right')` |
| Inner join | `pd.merge(left, right, on='col', how='inner')` |
| Outer join | `pd.merge(left, right, on='col', how='outer')` |
| Different column names | `pd.merge(left, right, left_on='a', right_on='b')` |
| Chain merges | `df.merge(...).merge(...)` |
| Select columns before merge | `df[['col1', 'col2']]` |

---

## ➡️ Next Tutorial

In **Tutorial 6: Time in the Quarry**, you will learn to work with dates and times—parsing date strings, extracting components, and analyzing trends over time using the price records from the Capital markets.

*"The wharver price spiked on March 23rd," the trader said. "Live specimen. First one ever brought to the Capital alive."*

*"What happened to it?"*

*"Burned through its cage on the second night. Steam from its gills set the warehouse on fire. Three dead."*

*"And the price after that?"*

*"Higher. Much higher. Senators don't scare easily when they smell profit."*

---

## 💾 Save Your Work

**File > Save a copy in Drive** to keep your completed notebook.